In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler


# Load your dataset (Replace 'your_data.csv' with the name of your data file)
data = pd.read_csv('outputs/Final_Merged_Analyzing_Data.csv')

# drop all columns where retrun is nana
data = data.dropna(subset=['ret'])

y = data.ret

In [2]:
#print(y)

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    PolynomialFeatures,
    StandardScaler,
)
from sklearn.svm import SVR
from sklearn.feature_selection import RFECV
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline 
from sklearn.impute import SimpleImputer
from df_after_transform import df_after_transform
from sklearn.model_selection import KFold, cross_validate, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso
from sklearn import set_config
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

set_config(display="diagram")  # display='text' is the default

pd.set_option('display.max_colwidth', 1000, 'display.max_rows', 50, 'display.max_columns', None) 


## Winsorizing Outliers

In [4]:
less_na_data = data.interpolate(method='linear', limit_direction='both')
from outlier_report import outlier_report
vars_to_check = less_na_data .select_dtypes(include=['int', 'float']).columns.tolist()
vars_with_big_outliers = list(outlier_report(less_na_data,vars_to_check,thres=4,return_df=True,no_print=True)
                              .query('max_z_abs > 5').index)
from winsorizer_with_missing import winsorizer_with_missing
## just some variables from the list 
winsorized_data = winsorizer_with_missing(less_na_data, cols=vars_with_big_outliers)

In [5]:
processed_winsorized_data = winsorized_data.replace([np.inf, -np.inf], np.nan)
processed_winsorized_data = processed_winsorized_data.fillna(method='ffill')
final_processed_winsorized_data = processed_winsorized_data.dropna()

In [6]:
top_5 = final_processed_winsorized_data['ret'].quantile(0.95)
bottom_5 = final_processed_winsorized_data['ret'].quantile(0.05)
final_processed_winsorized_data = final_processed_winsorized_data[final_processed_winsorized_data['ret'] < top_5]
final_processed_winsorized_data = final_processed_winsorized_data[final_processed_winsorized_data['ret'] > bottom_5]

In [7]:
data = final_processed_winsorized_data.drop('ret',axis=1)

## Part 1: Preprocessing the data

In [8]:
#data.describe()

In [9]:
## create test set for use later - notice the (random_state=rng)
rng = np.random.RandomState(0)
# Split the data into train and test sets
for train_index, test_index in tscv.split(processed_winsorized_data):
    X_train, X_test = processed_winsorized_data.iloc[train_index], processed_winsorized_data.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [10]:
num_cols = ["BESG ESG Score", "BESG Environmental Pillar Score", 
            "BESG Governance Pillar Score", "BESG Social Pillar Score", 
            "S&P Global ESG Rank", "S&P Global Governance & Economic Dimension Rank", 
            "S&P Global Environmental Dimension Rank", "S&P Global Social Dimension Rank", 
            "yf_ESG_Score", "yf_E_Score", "yf_S_Score", "yf_G_Score", 
            "Overridable Adjusted Beta", "Overridable Alpha", 
            "Volatility 30 Day", #"Current Market Cap", "Historical Market Cap", 
            "Price Earnings Ratio (P/E)", "Revenue", "Basic Earnings per Share"]




cat_cols = ['Credit Benchmark Credit Risk Indicator','Bloomberg Issuer Default Risk']

In [28]:
## set up pipeline to clean each type of variable (1 pipe per var type)

numer_pipe = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())

cat_pipe   = make_pipeline(OneHotEncoder(handle_unknown='ignore'))

# combine those pipes into "preprocess" pipe

preproc_pipe = make_column_transformer(  
    (numer_pipe, num_cols),
    (cat_pipe, cat_cols),
    remainder = 'drop' 
)

In [29]:
## from df_after_transform import df_after_transform

preproc_df = df_after_transform(preproc_pipe, X_train)
print(f'There are {preproc_df.shape[1]} columns in the preprocessed data.')
#preproc_df.describe().T.round(2)

There are 44 columns in the preprocessed data.


In [30]:
## from sklearn.linear_model import Lasso

clf = Lasso(alpha = 0.3)
clf_pipe = make_pipeline(preproc_pipe, clf)

In [31]:
clf_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['BESG ESG Score',
                                                   'BESG Environmental Pillar '
                                                   'Score',
                                                   'BESG Governance Pillar '
                                                   'Score',
                                                   'BESG Social Pillar Score',
                                                   'S&P Global ESG Rank',
                                                   'S&P Global Governance & '
                                                   'Economic Dimension Rank',
                                                   'S&P Global E...
                                                   'yf_ESG_Score', 'yf_E_Score',
                                                   'yf_S_Score', 'yf_G_Score',
                                                   'Overridable Adjusted Beta',
                                                   'Overridable Alpha',
                                                   'Volatility 30 Day',
                                                   'Price Earnings Ratio (P/E)',
                                                   'Revenue',
                                                   'Basic Earnings per Share']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Credit Benchmark Credit '
                                                   'Risk Indicator',
                                                   'Bloomberg Issuer Default '
                                                   'Risk'])])),
                ('lasso', Lasso(alpha=0.3))])

In [ ]:
#print(final_processed_winsorized_data.dtypes)

## Optimize hyperparameters

In [37]:
from sklearn.linear_model import Lasso

lasso_pipe = make_pipeline(preproc_pipe,
                           #PolynomialFeatures(degree=2, interaction_only=True),
                           #TruncatedSVD(n_components=20, random_state=rng),
                           Lasso(alpha=.3))


In [38]:
search_alphas = list(np.linspace(0.0001,0.001,10))+\
                list(np.linspace(0.001,0.007,7))+\
                list(np.linspace(0.0072,0.0082,101))
                
parameters = [ {#'polynomialfeatures__degree':polynomialfeatures__degree,
                #'pca__n_components':pca__n_components,
                #'lasso__max_iter':search_max_iter ,
                'lasso__alpha': search_alphas
              }]



In [39]:
# set up search grid

grid_search = GridSearchCV(estimator = lasso_pipe, 
                           param_grid = parameters,
                           scoring = ['r2','neg_root_mean_squared_error','neg_mean_absolute_error'],
                           cv = KFold(10),
                           refit='r2')

In [41]:
results = grid_search.fit(X_train,y_train)

In [42]:
df = pd.DataFrame(results.cv_results_).set_index('params')

In [43]:
notordereddf = df[['mean_test_r2', 'std_test_r2', 'rank_test_r2',
    'mean_test_neg_root_mean_squared_error', 'std_test_neg_root_mean_squared_error', 'rank_test_neg_root_mean_squared_error',
    'mean_test_neg_mean_absolute_error','std_test_neg_mean_absolute_error','rank_test_neg_mean_absolute_error']]

In [44]:
best_pipe = grid_search.best_estimator_ 
best_pipe.fit(X_train,y_train)
y_test_predict = best_pipe.predict(X_test)

In [45]:
# # score it on the test sample:
r2_test_score = r2_score(y_test,y_test_predict)
rmse_test_score = mean_squared_error(y_test, y_test_predict, squared=False)
rae_test_score = mean_absolute_error(y_test, y_test_predict) / abs(y_test.mean())

## Scoring Review

In [46]:
print(f'r2_test_score: {r2_test_score:.5f}')
print(f'rmse_test_score: {rmse_test_score:.5f}')
print(f'rae_test_score: {rae_test_score:.5f}')

r2_test_score: 0.00151
rmse_test_score: 0.19150
rae_test_score: 10.20815
